## **Importing Libraries**

In [1]:
!pip install groq langgraph langchain_community chromadb sentence-transformers langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.9 MB/s eta 0:0

In [2]:
from google.colab import userdata
from groq import Groq
from typing import Annotated, Literal
from typing_extensions import TypedDict

from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

## **Initializing Groq client**

In [3]:
GROQ_API_KEY = userdata.get('groqApiKey')
client = Groq(api_key=GROQ_API_KEY)

In [4]:
SOURCE_URL = "https://joinseven.medium.com/blog-series-genai-a-brief-introduction-in-generative-ai-4e11154df3f2"
loader = WebBaseLoader(web_paths=[SOURCE_URL])
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://joinseven.medium.com/blog-series-genai-a-brief-introduction-in-generative-ai-4e11154df3f2', 'title': 'Just a moment...', 'language': 'en-US'}, page_content='Just a moment...Enable JavaScript and cookies to continue\n')]


## **Splitting document into chunk**

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(docs)

## **Storing in Chromadb**

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(split_docs, embeddings)

/tmp/ipython-input-291995119.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **Retriever**

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

## **State Definition**

In [8]:
class MessageClassifier(BaseModel):
  message_type: Literal["rag","logical"] = Field(
  ...,
  description="Classify if the message requires retrieval (rag) or a direct logical response."
  )

class State(TypedDict):
  messages: Annotated[list, add_messages]
  message_type: str | None

## **Classifier Node**

In [9]:
def classify_messages(state: State):
    last_message = state["messages"][-1]
    resp = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system",
                "content": (
                    "Classify the user message as 'rag' if it likely benefits from external context "
                    "(e.g., asks about topics in the scraped document or requests factual info). "
                    "Otherwise classify as 'logical'. Reply with only 'rag' or 'logical'."
                )
            },
            {"role": "user", "content": last_message.content}
        ],
        temperature=0
    )
    label = resp.choices[0].message.content.strip().lower()
    if "rag" in label:
        return {"message_type": "rag"}
    elif "logical" in label:
        return {"message_type": "logical"}
    else:
        return {"message_type": "logical"}

## **RAG Agent**

In [10]:
def rag_agent(state: State):
  last_message = state["messages"][-1]
  retreived_docs = retriever.invoke(last_message.content)
  context = "\n\n".join([doc.page_content for doc in retrieved_docs[:3]]) or "No relevant context found."

  resp = client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {
              "role":"system",
              "content":(
                  "You are a helpful assistant. Use the provided context to answer the user's question. "
                  "If context is insufficient, say so briefly and respond with best-effort clarity."
              )
          },
          {
              "role":"user",
              "content":f"Question: {last_message.content}\n\nContext:\n{context}"
          }
      ],
      temperature=0
  )

  reply = resp.choices[0].message.content.strip()
  return {"messages": [{"role": "assistant", "content": reply}]}

## **Logical Agent**

In [11]:
def logical_agent(state: State):
  last_message = state["messages"][-1]

  resp = client.chat.completions.create(
      model="llama-3.1-8b-instant",
      messages=[
          {
              "role":"system",
              "content":"You are a concise, factual assistant. Answer clearly and directly."

          },
          {
              "role":"user",
              "content":last_message.content
          }
      ],
      temperature=0
  )

  reply = resp.choices[0].message.content.strip()
  return {"messages": [{"role": "assistant", "content": reply}]}

## **Graph Builder with LangGraph**

In [12]:
graph_builder = StateGraph(State)

graph_builder.add_node("classifier", classify_messages)
graph_builder.add_node("rag_agent", rag_agent)
graph_builder.add_node("logical_agent", logical_agent)

graph_builder.add_edge(START, "classifier")

graph_builder.add_conditional_edges(
    "classifier",
    lambda state: state.get("message_type"),
    {
        "rag": "rag_agent",
        "logical": "logical_agent"
    }
)

graph_builder.add_edge("rag_agent", END)
graph_builder.add_edge("logical_agent", END)

graph = graph_builder.compile()

In [13]:
def run_chatbot():
    state = {"messages": [], "message_type": None}
    print("Type 'exit' to quit.")
    while True:
        user_input = input("Message: ")
        if user_input.strip().lower() == "exit":
            print("Bye")
            break
        state["messages"] = state.get("messages", []) + [{"role": "user", "content": user_input}]
        state = graph.invoke(state)
        if state.get("messages"):
            last = state["messages"][-1]
            if hasattr(last, "content"):
                print(f"Assistant: {last.content}")
                print("\n")
            elif isinstance(last, dict):
                print(f"Assistant: {last['content']}")
                print("\n")
            else:
                print(f"Assistant: {last}")
                print("\n")


In [14]:
run_chatbot()

Type 'exit' to quit.
Message: what is gen ai
Assistant: Generalized Artificial Intelligence (Gen AI) refers to a hypothetical AI system that possesses a broad range of cognitive abilities, similar to human intelligence. It would be capable of:

1. Reasoning and problem-solving across various domains
2. Learning from experience and adapting to new situations
3. Understanding and generating human-like language
4. Recognizing and interpreting visual and auditory inputs
5. Exhibiting creativity, intuition, and common sense

Gen AI would be a highly advanced, general-purpose AI that can perform a wide range of tasks, from simple to complex, without being limited to a specific domain or task. It would be able to learn, reason, and apply knowledge across various disciplines, much like a human expert.

However, the development of Gen AI is still in its infancy, and significant scientific and engineering challenges need to be overcome before such a system can be created.


Message: im feeling s